# Function for subtracting 2 models that have frequency, Sparameter or frequency,power data

In [1]:
import pandas 
import numpy as np
import os
from pyMeasure import *
import re
from scipy.stats.mstats import gmean

Importing Code.Utils.Names
Importing Code.DataHandlers.NISTModels
Importing Code.DataHandlers.GeneralModels
Importing Code.DataHandlers.TouchstoneModels
Importing Code.DataHandlers.XMLModels
Importing Code.DataHandlers.RadiCALModels
Importing Code.DataHandlers.ZipModels
Importing Code.DataHandlers.Translations
Importing Code.DataHandlers.StatistiCALModels
Importing Code.DataHandlers.MUFModels
Importing Code.Analysis.SParameter
Importing Code.InstrumentControl.Instruments
Importing Code.InstrumentControl.Experiments


In [18]:
def frequency_model_collapse_multiple_measurements(model,**options):
    """Returns a model with a single set of frequencies. Default is to average values together
    but geometric mean, std and median are options. Geometric means of odd number of negative values fails"""
    defaults={"method":"mean"}
    # load other options from model
    for option,value in model.options.iteritems():
        if not re.search('begin_line|end_line',option):
            defaults[option]=value
    defaults["header"]=model.header
    defaults["column_names"]=model.column_names
    defaults["footer"]=model.footer
    defaults["metadata"]=model.metadata

    collapse_options={}
    for key,value in defaults.iteritems():
        collapse_options[key]=value
    for key,value in options.iteritems():
        collapse_options[key]=value    
    unique_frequency_list=sorted(list(set(model["Frequency"])))
    frequency_selector=model.column_names.index("Frequency")
    out_data=[]
    for index, frequency in enumerate(unique_frequency_list):
        data_row=filter(lambda x: x[frequency_selector]==frequency,model.data)
        if re.search('mean|av',collapse_options["method"],re.IGNORECASE):
            new_row=np.mean(np.array(data_row),axis=0).tolist()
        elif re.search('median',collapse_options["method"],re.IGNORECASE):
            new_row=np.median(np.array(data_row),axis=0).tolist()
        elif re.search('geometric',collapse_options["method"],re.IGNORECASE):
            new_row=gmean(np.array(data_row),axis=0).tolist()
        elif re.search('st',collapse_options["method"],re.IGNORECASE):
            new_row=np.std(np.array(data_row),axis=0).tolist()
        
        out_data.append(new_row)

    collapse_options["data"]=out_data

    if collapse_options["specific_descriptor"]:
        collapse_options["specific_descriptor"]=collapse_options["method"]+"_"+\
        collapse_options["specific_descriptor"]
        
    
    resulting_model=AsciiDataTable(None,**collapse_options)
    return resulting_model

def frequency_model_difference(model_1,model_2,**options):
    """Takes the difference of two models that both have frequency and a similar set of columns. Returns an object that is 
    a list of [[frequency,column_1,..column_n],...] where columns are the same in the models. If  a particular subset of 
    columns is desired use columns=["Frequency","magS11] models can be any subclass of AsciiDataTable, SNP, or 
    pandas.DataFrame, if a column is a non-numeric type it drops it. The frequency list should be unique 
    (no multiple frequencies) for at least one model"""
    # Set up defaults and pass options
    defaults={"columns":"all","interpolate":False,"average":True}
    difference_options={}
    for key,value in defaults.iteritems():
        difference_options[key]=value
    for key,value in options.iteritems():
        difference_options[key]=value
        
    # first check type, if it is a panadas data frame a little conversion is needed, else is for all other models
    if type(model_1) in [pandas.DataFrame]:
        model_1=DataFrame_to_AsciiDataTable(model_1)
    if type(model_2) in [pandas.DataFrame]:
        model_2=DataFrame_to_AsciiDataTable(model_2)
    # now start with a set of frequencies (unique values from both)
    frequency_set_1=set(model_1["Frequency"])
    frequency_set_2=set(model_2["Frequency"])
    model_2_frequency_selector=model_2.column_names.index('Frequency')
    column_names_set_1=set(model_1.column_names)
    column_names_set_2=set(model_2.column_names)
    

    # All points must be in the intersection to be used
    frequency_intersection=list(frequency_set_1.intersection(frequency_set_2))
    column_names_intersection=list(column_names_set_1.intersection(column_names_set_2))
    
    if not frequency_intersection:
        print("The models do not have any frequency points in common")
        return None
    
    difference_data=[]
    for row_index,frequency in enumerate(model_1["Frequency"]):
        new_row=[frequency]
        new_column_names=["Frequency"]
        if frequency in frequency_intersection:
            model_2_frequency_row=filter(lambda x: x[model_2_frequency_selector]==frequency,model_2.data)[0]
            #print("{0} is {1}".format("model_2_frequency_row",model_2_frequency_row))
            for column_index,column in enumerate(model_1.column_names):
                if column in column_names_intersection and column not in ["Frequency"]:
                    model_2_column_selector=model_2.column_names.index(column)
                    if re.search('int|float',
                                 model_1.options["column_types"][column_index],
                                 re.IGNORECASE) and re.search('int|float',
                                                              model_2.options["column_types"][model_2_column_selector],
                                                              re.IGNORECASE):
                        
                        new_row.append(model_1.data[row_index][column_index]-model_2_frequency_row[model_2_column_selector])
                        new_column_names.append(column)
                    elif difference_options["columns"] in ["all"]:
                        new_row.append(model_1.data[row_index][column_index])
                        new_column_names.append(column)
            difference_data.append(new_row)
    difference_options["column_names"]=new_column_names
    difference_options["data"]=difference_data      
    result=AsciiDataTable(None,**difference_options)
    return result

In [3]:
# examples

COMBINED_RESULTS_FILE_DIRECTORY=r"C:\Users\sandersa\Desktop\Combined_Results_Files"
ONE_PORT_COMBINED_FILE_NAME='One_Port_Combined_Results_File.csv'

model_1=pandas.read_csv(os.path.join(COMBINED_RESULTS_FILE_DIRECTORY,ONE_PORT_COMBINED_FILE_NAME))


In [4]:
raw_one_port=OnePortRawModel(os.path.join(TESTS_DIRECTORY,'OnePortRawTestFile.txt'))


In [5]:
res_dataframe=model_1[model_1["Device_Id"]==raw_one_port.metadata["Device_Id"]]

In [6]:
difference=frequency_model_difference(raw_one_port,res_dataframe)

model_2_frequency_row is ['CTN112', 0.01, 6, 0.9999600000000001, -0.413]
model_2_frequency_row is ['CTN112', 0.01, 6, 0.9999600000000001, -0.413]
model_2_frequency_row is ['CTN112', 0.01, 6, 0.9999600000000001, -0.413]
model_2_frequency_row is ['CTN112', 0.05, 6, 1.00003, -2.266]
model_2_frequency_row is ['CTN112', 0.05, 6, 1.00003, -2.266]
model_2_frequency_row is ['CTN112', 0.05, 6, 1.00003, -2.266]
model_2_frequency_row is ['CTN112', 0.1, 6, 1.00024, -4.5280000000000005]


In [7]:
import matplotlib.pyplot as plt
plt.plot(difference["Frequency"],difference["magS11"])
plt.show()

In [8]:
print difference

Frequency,magS11,argS11
0.01,-0.00036,-0.057
0.01,-0.00036,-0.067
0.01,-0.00036,-0.067
0.05,-0.00023,0.016
0.05,-0.00023,0.016
0.05,-0.00023,0.016
0.1,-0.00044,0.038


In [19]:
diff_2=frequency_model_difference(res_dataframe,res_dataframe)

In [20]:
print diff_2

Frequency,Device_Id,Number_Measurements,magS11,argS11
0.01,CTN112,0,0.0,0.0
0.05,CTN112,0,0.0,0.0
0.1,CTN112,0,0.0,0.0
0.2,CTN112,0,0.0,0.0
0.4,CTN112,0,0.0,0.0
0.6,CTN112,0,0.0,0.0
0.8,CTN112,0,0.0,0.0
1.0,CTN112,0,0.0,0.0
1.2,CTN112,0,0.0,0.0
1.4,CTN112,0,0.0,0.0
1.6,CTN112,0,0.0,0.0
1.8,CTN112,0,0.0,0.0
2.0,CTN112,0,0.0,0.0
2.2,CTN112,0,0.0,0.0
2.4,CTN112,0,0.0,0.0
2.6,CTN112,0,0.0,0.0
2.8,CTN112,0,0.0,0.0
3.0,CTN112,0,0.0,0.0
3.2,CTN112,0,0.0,0.0
3.4,CTN112,0,0.0,0.0
3.6,CTN112,0,0.0,0.0
3.8,CTN112,0,0.0,0.0
4.0,CTN112,0,0.0,0.0
4.2,CTN112,0,0.0,0.0
4.4,CTN112,0,0.0,0.0
4.6,CTN112,0,0.0,0.0
4.8,CTN112,0,0.0,0.0
5.0,CTN112,0,0.0,0.0
5.2,CTN112,0,0.0,0.0
5.4,CTN112,0,0.0,0.0
5.6,CTN112,0,0.0,0.0
5.8,CTN112,0,0.0,0.0
6.0,CTN112,0,0.0,0.0
6.2,CTN112,0,0.0,0.0
6.4,CTN112,0,0.0,0.0
6.6,CTN112,0,0.0,0.0
6.8,CTN112,0,0.0,0.0
7.0,CTN112,0,0.0,0.0
7.2,CTN112,0,0.0,0.0
7.4,CTN112,0,0.0,0.0
7.6,CTN112,0,0.0,0.0
7.8,CTN112,0,0.0,0.0
8.0,CTN112,0,0.0,0.0
8.2,CTN112,0,0.0,0.0
8.4,CTN112,0,0.0,0.0

In [11]:
map(lambda x:str(x),model_1.dtypes.tolist())

['object', 'float64', 'int64', 'float64', 'float64']

In [12]:
def pandas_to_ascii_data_table(pandas_data_frame,**options):
    """Converts a pandas.DataFrame to an AsciiDataTable"""
    # Set up defaults and pass options
    defaults={}
    conversion_options={}
    for key,value in defaults.iteritems():
        conversion_options[key]=value
    for key,value in options.iteritems():
        conversion_options[key]=value
        
    conversion_options["column_names"]=pandas_data_frame.columns.tolist()
    conversion_options["data"]=pandas_data_frame.as_matrix().tolist()
    conversion_options["column_types"]=map(lambda x:str(x),pandas_data_frame.dtypes.tolist())
    
    new_table=AsciiDataTable(None,conversion_options)
    return new_table

In [13]:
def DataFrame_to_AsciiDataTable(pandas_data_frame,**options):
    """Converts a pandas.DataFrame to an AsciiDataTable"""
    # Set up defaults and pass options
    defaults={}
    conversion_options={}
    for key,value in defaults.iteritems():
        conversion_options[key]=value
    for key,value in options.iteritems():
        conversion_options[key]=value
        
    conversion_options["column_names"]=pandas_data_frame.columns.tolist()
    conversion_options["data"]=pandas_data_frame.as_matrix().tolist()
    conversion_options["column_types"]=map(lambda x:str(x),pandas_data_frame.dtypes.tolist())
   
    new_table=AsciiDataTable(None,**conversion_options)
    return new_table

In [14]:
from pyMeasure import *

In [15]:
test_model=DataFrame_to_AsciiDataTable(model_1)

In [16]:
test_model==DataFrame_to_AsciiDataTable(AsciiDataTable_to_DataFrame(test_model))

True

In [17]:
test_model.path

'Data_Table_20161115_001.txt'